## Measuring energy and energy variance with classical shadows

Related Paper [Huang et al, Nat Phys 2020](https://doi.org/10.1038/s41567-020-0932-7)

In [42]:
using Revise
using RandomMeas
using Statistics

# Ising Hamiltonian

We first construct the Hamiltonian of the transverse field Ising model with N qubits.

In [43]:
N = 8 # Number of qubits
ξ = siteinds("Qubit", N) # ITensor site indices for classical shadows
# Define the Hamiltonian
J = -1. # Ising interaction strength
B = 1. # Transverse field strength
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising XX interactions
  ampo .+= J , "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field Z
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ) # Hamiltonian as MPO
H2 = apply(H,H); # H^2 for later use

# Groundstate energy and variance

First, we compute the ground state $\psi_G$ of the transverse field Ising model with DMRG.

In [44]:
# Define the DMRG parameters 
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)

# Run DMRG to get the ground state
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψG = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-9.8241897622091  maxlinkdim=4 maxerr=2.47E-16 time=0.005
After sweep 2 energy=-9.837942407805736  maxlinkdim=11 maxerr=5.65E-11 time=0.018
After sweep 3 energy=-9.837951446540888  maxlinkdim=7 maxerr=9.45E-11 time=0.026
After sweep 4 energy=-9.837951446199035  maxlinkdim=6 maxerr=7.09E-11 time=0.005
After sweep 5 energy=-9.837951446199062  maxlinkdim=6 maxerr=6.91E-11 time=0.005

Ground state energy:  -9.837951446199062

---------------------------------------



We compute its energy and variance.

In [45]:
E_G = real(inner(ψG', H,ψG))
println("final energy ",E_G)
E2_G = real(inner(ψG', H2,ψG))
println("final expect of H2 ",E2_G)
Var_E_G = E2_G-E_G^2
println("final energy variance ",Var_E_G)

final energy -9.837951446199062
final expect of H2 96.78528867040481
final energy variance 1.2634600921046513e-8


### Estimation with classical shadows 

We estimate its energy and variance with classical shadows. First, we sample the measurement settings.

In [46]:
NU = 1000 # Number of random unitaries
measurement_settings=RandomMeas.LocalUnitaryMeasurementSettings(N,NU,site_indices = ξ);

Next, we simulate the randomized measurements.

In [47]:
NM = 100
measurement_results = simulate_RandomMeas(ψG,measurement_settings,NM,"dense");

In [48]:
factorized_shadows = get_factorized_shadows(measurement_results);

In [49]:
Es_m,Es_ste = get_expect_shadow(H,factorized_shadows,compute_sem=true)
E2s_m,E2s_ste = get_expect_shadow(H2,factorized_shadows,compute_sem=true)
Es_m = real(Es_m)
E2s_m = real(E2s_m)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy of the circuit generated state ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2 of the circuit generated state ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance of the circuit generated state ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.

Estimated energy of the circuit generated state -9.82±0.03
Estimated expectation value of H^2 of the circuit generated state 96.6±0.5
Estimated energy variance of the circuit generated state 0.1±0.07


# Output state of quantum circuit

We construct a state as output of a quantum circuit.

In [50]:
η = 10 # Number of layer blocks
m = 4 # Number of layers per block
# Build the gate structure
circuit = ITensor[]
for d in 1:η
    xx_layer = [op("Rxx", ξ[j], ξ[j+1], ϕ=1*d/η/m) for j in 1:(N - 1)] # Rxx gates TODO: PastaQ dependence removed
    z_layer = [op("Rz", ξ[j], θ=2*B/m) for j in 1:N] # Rxx gates TODO: PastaQ dependence removed
    for t in 1:m
        append!(circuit, xx_layer)
        append!(circuit, z_layer)
    end
end
ψ0 = MPS(ξ,["Up" for n in 1:N]);
ψt = apply(circuit,ψ0; cutoff=1E-8)
normalize!(ψt) # Normalize the state TODO: Is this ok?
#orthogonalize!(ψt,1) TODO: why do we need this? 
#ψt[1] /= norm(ψt[1]) TODO: why do we need this?


MPS
[1] ((dim=2|id=854|"Qubit,Site,n=1"), (dim=2|id=777|"Link,n=1"))
[2] ((dim=2|id=589|"Qubit,Site,n=2"), (dim=2|id=777|"Link,n=1"), (dim=4|id=540|"Link,n=1"))
[3] ((dim=2|id=30|"Qubit,Site,n=3"), (dim=4|id=540|"Link,n=1"), (dim=8|id=58|"Link,n=1"))
[4] ((dim=2|id=956|"Qubit,Site,n=4"), (dim=8|id=58|"Link,n=1"), (dim=13|id=88|"Link,n=1"))
[5] ((dim=2|id=399|"Qubit,Site,n=5"), (dim=13|id=88|"Link,n=1"), (dim=8|id=302|"Link,n=1"))
[6] ((dim=2|id=548|"Qubit,Site,n=6"), (dim=8|id=302|"Link,n=1"), (dim=4|id=586|"Link,n=1"))
[7] ((dim=2|id=322|"Qubit,Site,n=7"), (dim=4|id=586|"Link,n=1"), (dim=2|id=133|"Link,n=1"))
[8] ((dim=2|id=585|"Qubit,Site,n=8"), (dim=2|id=133|"Link,n=1"))


We compute its energy and variance.

In [51]:
E_th = real(inner(ψt', H,ψt)) # Energy of ψt TODO: I don't understand the naming convention here. What does the _th stand for? Thermal?
println("final energy ",E_th)
Var_E_th = real(inner(ψt', H2,ψt))-E_th^2 # Energy variance of ψt
println("final energy variance ",Var_E_th)

final energy -9.47182617618307
final energy variance 1.6950453000525556


In [52]:
NU = 1000 # Number of random unitaries
measurement_settings=RandomMeas.LocalUnitaryMeasurementSettings(N,NU,site_indices = ξ);

NM = 400 # Number of measurements per unitary
measurement_results = simulate_RandomMeas(ψG,measurement_settings,NM,"dense");

In [54]:
factorized_shadows = get_factorized_shadows(measurement_results);

In [ ]:
Es_m,Es_ste = get_expect_shadow(H,factorized_shadows,compute_sem=true)
E2s_m,E2s_ste = get_expect_shadow(H2,factorized_shadows,compute_sem=true)
Es_m = real(Es_m)
E2s_m = real(E2s_m)
Evar = E2s_m .- Es_m.^2
Evar_ste = Evar*sqrt(4*Es_ste^2*Es_m^2 + E2s_ste^2)
println("Estimated energy of the circuit generated state ", round(Es_m,digits=2), "±", round(Es_ste, sigdigits=1))
println("Estimated expectation value of H^2 of the circuit generated state ", round(E2s_m,digits=1), "±", round(E2s_ste, sigdigits=1))
println("Estimated energy variance of the circuit generated state ", round(Evar,digits=1), "±", round(Evar_ste, sigdigits=1)) # We oversimplify here. E2s_m and E_m are not independent.